In [116]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [117]:
import pandas as pd
from glob import glob 
from fastcore.xtras import load_pickle

from gpt3forchem.output import get_regression_metrics
from gpt3forchem.api_wrappers import extract_prediction

from pycm import ConfusionMatrix

## Classification

In [118]:
all_res = glob('results/20221130_esol/*')

In [119]:
metrics = []
baselines = []

for res in all_res:
    res = load_pickle(res)
    cm = res['cm']
    cm = ConfusionMatrix(list(map(lambda x: str(x).strip(), res['cm'].actual_vector)), list(map(lambda x: str(x).strip(), res['cm'].predict_vector)))
    baseline_cm = res['baseline']['cm']
    baseline_cm = ConfusionMatrix(list(map(lambda x: str(x).strip(), baseline_cm.actual_vector)), list(map(lambda x: str(x).strip(), baseline_cm.predict_vector)))
    metrics.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': cm.ACC_Macro,
            'f1_macro': cm.F1_Macro,
            'f1_micro': cm.F1_Micro
        })
    baselines.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'accuracy': baseline_cm.ACC_Macro,
            'f1_macro': baseline_cm.F1_Macro,
            'f1_micro': baseline_cm.F1_Micro  
        })

In [120]:
res = load_pickle(all_res[-1])

In [121]:
metrics = pd.DataFrame(metrics)
baselines = pd.DataFrame(baselines)

In [122]:
metrics.groupby(['representation', 'train_size']).agg(['mean', 'std', 'count'])

accuracy                  f1_macro                  \
                               mean       std count      mean       std count   
representation train_size                                                       
inchi          10          0.750667  0.021939     3  0.126942  0.040486     3   
               50          0.796667  0.027227     3  0.213825  0.092108     3   
               500         0.899000  0.004243     2  0.622984  0.055124     2   
iupac_name     10          0.756000  0.025456     2  0.150165  0.067264     2   
               50          0.774000  0.005657     2  0.184889  0.025109     2   
               500         0.886000       NaN     1  0.634010       NaN     1   
selfies        10          0.785083  0.029413     4  0.162095  0.050342     4   
               50          0.787000  0.017626     4  0.227685  0.044624     4   
               500         0.877333  0.003055     3  0.617188  0.029482     3   
smiles         10          0.757200  0.031515     5  0.136093  0.047727     5   
               50          0.789000  0.017474     4  0.331951  0.069519     4   
               500         0.892000  0.000000     4  0.667950  0.022826     4   

                           f1_micro                  
                               mean       std count  
representation train_size                            
inchi          10          0.376667  0.054848     3  
               50          0.491667  0.068069     3  
               500         0.747500  0.010607     2  
iupac_name     10          0.390000  0.063640     2  
               50          0.435000  0.014142     2  
               500         0.715000       NaN     1  
selfies        10          0.438750  0.063163     4  
               50          0.467500  0.044064     4  
               500         0.693333  0.007638     3  
smiles         10          0.393000  0.078788     5  
               50          0.472500  0.043684     4  
               500         0.730000  0.000000     4

In [123]:
baselines.groupby(['train_size']).agg(['mean', 'std'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_65015/4006146815.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  baselines.groupby(['train_size']).agg(['mean', 'std'])


accuracy            f1_macro            f1_micro          
                mean       std      mean       std      mean       std
train_size                                                            
10          0.763857  0.008018  0.116184  0.004229  0.409643  0.020045
50          0.852000  0.000000  0.368690  0.000000  0.630000  0.000000
500         0.818182  0.000000  0.000000  0.000000  0.000000  0.000000

## Regression

In [124]:
all_res_regression = glob('results/20221129_esol_regression/*')

In [125]:
metrics_regression = []
baselines_regression = []

for res in all_res_regression:
    res = load_pickle(res)
    metrics_regression.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'r2': res['metrics']['r2'],
            'max_error': res['metrics']['max_error'],
            'mean_absolute_error': res['metrics']['mean_absolute_error'],
            'mean_squared_error': res['metrics']['mean_squared_error'],
            'rmse': res['metrics']['rmse'],
        })
    baselines_regression.append(
        {
            'train_size': res['train_size'],
            'representation': res['representation'],
            'r2': res['baseline']['r2'],
            'max_error': res['baseline']['max_error'],
            'mean_absolute_error': res['baseline']['mean_absolute_error'], 
            'mean_squared_error': res['baseline']['mean_squared_error'],
            'rmse': res['baseline']['rmse'],
        })

In [126]:
metrics_regression = pd.DataFrame(metrics_regression)

baselines_regression = pd.DataFrame(baselines_regression)

In [127]:
metrics_regression.groupby(['representation', 'train_size']).agg(['mean', 'std', 'count'])

r2                 max_error                  \
                               mean       std count      mean       std count   
representation train_size                                                       
inchi          10         -0.113511  0.284540     2  6.440000  1.640488     2   
               50          0.184728  0.183590     2  5.480000  0.707107     2   
               500         0.697506  0.056540     2  4.700000  1.923330     2   
iupac_name     10         -0.685451  0.015444     2  5.980000  0.000000     2   
               50         -0.307868  0.134366     2  8.430000  0.141421     2   
               500         0.564888  0.037310     2  6.320000  0.989949     2   
selfies        10         -0.485522  0.342497     3  5.576667  0.561456     3   
               50         -0.184180  0.154835     3  7.353333  0.041633     3   
               500         0.712676  0.025338     2  3.475000  0.190919     2   
smiles         10         -0.213162  0.257878     6  6.503333  0.744222     6   
               50          0.026252  0.124657     5  6.756000  1.061475     5   
               500         0.812973  0.012831     4  2.842500  0.192592     4   

                          mean_absolute_error                  \
                                         mean       std count   
representation train_size                                       
inchi          10                    1.645450  0.194172     2   
               50                    1.416550  0.168079     2   
               500                   0.789050  0.026375     2   
iupac_name     10                    2.191200  0.005020     2   
               50                    1.700925  0.111829     2   
               500                   0.902750  0.021779     2   
selfies        10                    2.030950  0.272153     3   
               50                    1.675750  0.130111     3   
               500                   0.798350  0.039739     2   
smiles         10                    1.742408  0.216714     6   
               50                    1.515210  0.097677     5   
               500                   0.656112  0.032815     4   

                          mean_squared_error                      rmse  \
                                        mean       std count      mean   
representation train_size                                                
inchi          10                   4.459442  1.139537     2  2.103031   
               50                   3.362220  0.757135     2  1.827777   
               500                  1.171750  0.219014     2  1.080098   
iupac_name     10                   6.749972  0.061850     2  2.598057   
               50                   5.393711  0.554132     2  2.320902   
               500                  1.685464  0.144526     2  1.297657   
selfies        10                   5.949288  1.371647     3  2.428841   
               50                   4.883616  0.638546     3  2.206746   
               500                  1.112986  0.098151     2  1.054468   
smiles         10                   4.822443  1.069518     6  2.185279   
               50                   3.988145  0.522323     5  1.993845   
               500                  0.724472  0.049704     4  0.850786   

                                           
                                std count  
representation train_size                  
inchi          10          0.270927     2  
               50          0.207119     2  
               500         0.101386     2  
iupac_name     10          0.011903     2  
               50          0.119379     2  
               500         0.055687     2  
selfies        10          0.273916     3  
               50          0.144336     3  
               500         0.046540     2  
smiles         10          0.237478     6  
               50          0.126130     5  
               500         0.029120     4

In [128]:
baselines_regression.groupby(['train_size']).agg(['mean', 'std', 'count'])

/var/folders/m9/_txh68y946s4pxy1x2wnd3lh0000gn/T/ipykernel_65015/3958601162.py:1: FutureWarning: ['representation'] did not aggregate successfully. If any error is raised this will raise in a future version of pandas. Drop these columns/ops to avoid this warning.
  baselines_regression.groupby(['train_size']).agg(['mean', 'std', 'count'])


r2                 max_error                  \
                mean       std count      mean       std count   
train_size                                                       
10          0.010333  0.030110    13  5.634251  0.204666    13   
50          0.594279  0.017111    12  4.406837  0.165991    12   
500         0.896202  0.000000    10  1.975797  0.000000    10   

           mean_absolute_error                 mean_squared_error            \
                          mean       std count               mean       std   
train_size                                                                    
10                    1.549004  0.020711    13           3.737158  0.083279   
50                    0.953239  0.022597    12           1.604161  0.096878   
500                   0.492260  0.000000    10           0.437248  0.000000   

                      rmse                  
           count      mean       std count  
train_size                                  
10            13  1.933059  0.021871    13  
50            12  1.265973  0.040095    12  
500           10  0.661248  0.000000    10